In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import windows
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings

warnings.simplefilter("ignore", category=RuntimeWarning)

# === Config ===
FILE_PATH = "dataset/raw_sample/GDN_6_s3_v1_003_D.npy"
RX_INDEX = 0
TARGET_FRAME = 26  # 26번 프레임만 표시
SAVE_DIR = "rdi_outputs"
os.makedirs(SAVE_DIR, exist_ok=True)

# === RDI 계산 함수 ===
def compute_single_rdi(frames, rx_index=0, frame_idx=0):
    """특정 프레임의 RDI만 계산"""
    raw_frame = frames[frame_idx, :, :, rx_index]

    # === 2D 평균 제거 ===
    col_mean = np.mean(raw_frame, axis=0, keepdims=True)
    mean_sub = raw_frame - col_mean
    row_mean = np.mean(mean_sub, axis=1, keepdims=True)
    mean_sub = mean_sub - row_mean

    # === Range FFT ===
    X = mean_sub * windows.hamming(mean_sub.shape[1])[None, :]
    range_fft = np.fft.fft(X, axis=1, n=448)[:, :224]

    # === Doppler FFT ===
    doppler_window = windows.hamming(range_fft.shape[0])[:, None]
    doppler_input = range_fft * doppler_window
    doppler_fft = np.fft.fftshift(np.fft.fft(doppler_input, axis=0), axes=0)

    # === Magnitude + log1p ===
    return np.log1p(np.abs(doppler_fft))


# === 데이터 로드 ===
frames = np.load(FILE_PATH)  # (50, 3, 256, 128)
frames = np.transpose(frames, (0, 2, 3, 1))  # (50, 256, 128, 3)
print(f"Loaded frames: {frames.shape}")

# === 26번 프레임 RDI 계산 ===
rdi_26 = compute_single_rdi(frames, RX_INDEX, TARGET_FRAME)

# === 시각화 및 저장 ===
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(rdi_26, cmap='turbo', origin='lower', aspect='auto')
ax.axis('off')  # 축, 제목 제거

# === 컬러바 (눈금만 표시, 축제목 제거) ===
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = fig.colorbar(im, cax=cax)
cbar.ax.tick_params(labelsize=18)
cbar.set_label("")

# === 여백 최소화 + 고해상도 저장 ===
plt.tight_layout(pad=0)
save_path = os.path.join(SAVE_DIR, f"RDI_frame{TARGET_FRAME}_rx{RX_INDEX}.png")
plt.savefig(save_path, dpi=600, bbox_inches='tight', pad_inches=0)
plt.close()

print(f"[Saved] {save_path}")


Loaded frames: (50, 256, 128, 3)
[Saved] rdi_outputs/RDI_frame26_rx0.png


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

# ===== CONFIG =====
PREPROCESS_DIR = "dataset/preprocessed_sample"   # .npy 파일이 있는 폴더
SAVE_DIR_RTM = "plots_rtm"
SAVE_DIR_DTM = "plots_dtm"
os.makedirs(SAVE_DIR_RTM, exist_ok=True)
os.makedirs(SAVE_DIR_DTM, exist_ok=True)

# ===== 시각화 함수 =====
def visualize_and_save_single_npy(npy_path, save_dir):
    """_rtm.npy 또는 _dtm.npy 파일 하나를 이미지로 저장"""
    arr = np.load(npy_path)  # (224, 224, 3)
    
    # RX0만 사용
    arr_rx0 = arr[:, :, 0]

    # 시각화 및 저장
    plt.figure(figsize=(5, 5))
    plt.imshow(arr_rx0, cmap='turbo', aspect='auto')
    plt.axis('off')
    plt.tight_layout(pad=0)

    base_id = os.path.splitext(os.path.basename(npy_path))[0]
    save_path = os.path.join(save_dir, f"{base_id}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()

    print(f"[Saved] {save_path}")

# ===== Main Loop =====
npy_paths = glob(os.path.join(PREPROCESS_DIR, "*.npy"))
print(f"[Info] Found {len(npy_paths)} npy files in {PREPROCESS_DIR}")

for npy_path in npy_paths:
    filename = os.path.basename(npy_path)
    
    if filename.endswith("_rtm.npy"):
        visualize_and_save_single_npy(npy_path, SAVE_DIR_RTM)
    elif filename.endswith("_dtm.npy"):
        visualize_and_save_single_npy(npy_path, SAVE_DIR_DTM)
    else:
        print(f"[Skip] {filename}: _rtm/_dtm이 아닌 파일")


[Info] Found 30 npy files in dataset/preprocessed_sample
[Saved] plots_dtm/GDN_0_s1_v4_026_G_dtm.png
[Saved] plots_rtm/GDN_0_s1_v4_026_G_rtm.png
[Saved] plots_dtm/GDN_10_s2_v3_020_G_dtm.png
[Saved] plots_rtm/GDN_10_s2_v3_020_G_rtm.png
[Saved] plots_dtm/GDN_11_s3_v4_022_N_dtm.png
[Saved] plots_rtm/GDN_11_s3_v4_022_N_rtm.png
[Saved] plots_dtm/GDN_12_s3_v4_026_D_dtm.png
[Saved] plots_rtm/GDN_12_s3_v4_026_D_rtm.png
[Saved] plots_dtm/GDN_13_s4_v4_029_N_dtm.png
[Saved] plots_rtm/GDN_13_s4_v4_029_N_rtm.png
[Saved] plots_dtm/GDN_14_s1_v2_016_G_dtm.png
[Saved] plots_rtm/GDN_14_s1_v2_016_G_rtm.png
[Saved] plots_dtm/GDN_1_s4_v4_026_N_dtm.png
[Saved] plots_rtm/GDN_1_s4_v4_026_N_rtm.png
[Saved] plots_dtm/GDN_2_s2_v2_011_D_dtm.png
[Saved] plots_rtm/GDN_2_s2_v2_011_D_rtm.png
[Saved] plots_dtm/GDN_3_s2_v1_003_G_dtm.png
[Saved] plots_rtm/GDN_3_s2_v1_003_G_rtm.png
[Saved] plots_dtm/GDN_4_s1_v1_002_N_dtm.png
[Saved] plots_rtm/GDN_4_s1_v1_002_N_rtm.png
[Saved] plots_dtm/GDN_5_s2_v2_013_G_dtm.png
[Saved] p

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

# ===== CONFIG =====
PREPROCESS_DIR = "dataset/preprocessed_sample"
SAVE_DIR = "plots_rx0"
os.makedirs(SAVE_DIR, exist_ok=True)

# ===== 시각화 및 저장 함수 =====
def save_with_axes(img, save_path, is_rtm=True):
    plt.figure(figsize=(3.8, 3.8))
    ax = plt.gca()

    # ---- 공통 x축 설정 ----
    x_max = 4.0  # 최대값 4초로 고정

    if is_rtm:
        # ---- RTM 설정 (Range) ----
        extent = [0, x_max, 0.0, 1.745]
        ax.imshow(img, cmap='turbo', origin='lower', extent=extent, aspect='auto')

        ax.invert_yaxis()

        # y축: 0, 0.5, 1.0, 1.5 m
        yticks_pos = [0.0, 0.5, 1.0, 1.5]
        ax.set_yticks(yticks_pos)
        ax.set_yticklabels(["0.0", "0.5", "1.0", "1.5"])
        ax.set_ylabel("Range [m]", fontsize=14, fontweight='bold', labelpad=2)

    else:
        # ---- DTM 설정 (Velocity) ----
        v_max = 4.115
        extent = [0, x_max, -v_max, v_max]
        ax.imshow(img, cmap='turbo', origin='upper', extent=extent, aspect='auto')

        ax.set_ylim(-v_max, v_max)
        ax.set_yticks([4, 2, 0, -2, -4])
        ax.set_yticklabels(["-4", "-2", "0", "2", "4"])
        ax.set_ylabel("Velocity (m/s)", fontsize=14, fontweight='bold', labelpad=2)

    # ---- X축: 0, 2, 4 ----
    ax.set_xlim(0, x_max)
    ax.set_xticks([0, 2, 4])
    ax.set_xlabel("Time [s]", fontsize=14, fontweight='bold', labelpad=2)

    ax.set_box_aspect(1)
    ax.tick_params(axis='both', which='major', labelsize=14, pad=2)
    

    plt.subplots_adjust(left=0.16, right=0.98, bottom=0.16, top=0.98)
    plt.savefig(save_path, dpi=600)
    plt.close()

# ===== Main 시각화 함수 =====
def visualize_rx0_and_save(base_id, rtm_path, dtm_path, save_dir):
    rtm = np.load(rtm_path)
    dtm = np.load(dtm_path)

    rtm_rx0 = rtm[:, :, 0]
    dtm_rx0 = dtm[:, :, 0]

    # ----- 저장 -----
    rtm_save_path = os.path.join(save_dir, f"{base_id}_rtm.png")
    dtm_save_path = os.path.join(save_dir, f"{base_id}_dtm.png")

    save_with_axes(rtm_rx0, rtm_save_path, is_rtm=True)
    save_with_axes(dtm_rx0, dtm_save_path, is_rtm=False)

    print(f"[Saved] {rtm_save_path}, {dtm_save_path}")

# ===== Main Loop =====
npy_files = glob(os.path.join(PREPROCESS_DIR, "*.npy"))
print(f"[Info] Found {len(npy_files)} npy files")

for npy_path in npy_files:
    base_id = os.path.splitext(os.path.basename(npy_path))[0]

    if base_id.endswith("_rtm"):
        dtm_candidate = npy_path.replace("_rtm.npy", "_dtm.npy")
        if os.path.exists(dtm_candidate):
            visualize_rx0_and_save(base_id.replace("_rtm", ""), npy_path, dtm_candidate, SAVE_DIR)
        else:
            print(f"[Skip] {base_id}: 대응되는 _dtm.npy 없음")


[Info] Found 30 npy files
[Saved] plots_rx0/GDN_0_s1_v4_026_G_rtm.png, plots_rx0/GDN_0_s1_v4_026_G_dtm.png
[Saved] plots_rx0/GDN_10_s2_v3_020_G_rtm.png, plots_rx0/GDN_10_s2_v3_020_G_dtm.png
[Saved] plots_rx0/GDN_11_s3_v4_022_N_rtm.png, plots_rx0/GDN_11_s3_v4_022_N_dtm.png
[Saved] plots_rx0/GDN_12_s3_v4_026_D_rtm.png, plots_rx0/GDN_12_s3_v4_026_D_dtm.png
[Saved] plots_rx0/GDN_13_s4_v4_029_N_rtm.png, plots_rx0/GDN_13_s4_v4_029_N_dtm.png
[Saved] plots_rx0/GDN_14_s1_v2_016_G_rtm.png, plots_rx0/GDN_14_s1_v2_016_G_dtm.png
[Saved] plots_rx0/GDN_1_s4_v4_026_N_rtm.png, plots_rx0/GDN_1_s4_v4_026_N_dtm.png
[Saved] plots_rx0/GDN_2_s2_v2_011_D_rtm.png, plots_rx0/GDN_2_s2_v2_011_D_dtm.png
[Saved] plots_rx0/GDN_3_s2_v1_003_G_rtm.png, plots_rx0/GDN_3_s2_v1_003_G_dtm.png
[Saved] plots_rx0/GDN_4_s1_v1_002_N_rtm.png, plots_rx0/GDN_4_s1_v1_002_N_dtm.png
[Saved] plots_rx0/GDN_5_s2_v2_013_G_rtm.png, plots_rx0/GDN_5_s2_v2_013_G_dtm.png
[Saved] plots_rx0/GDN_6_s3_v1_003_D_rtm.png, plots_rx0/GDN_6_s3_v1_003_D_